## Summary of Study Results

With the data downloads from Replica, create a single table with a row for each origin station and columns for “car trips to areas around other destination stations on the other lines” and another column for “train trips”

In [1]:
import pandas as pd
import geopandas as gpd
from siuba import *
import numpy as np

from calitp_data_analysis.sql import to_snakecase
from calitp_data_analysis import utils
from calitp_data_analysis import get_fs
import gcsfs as fs
fs = get_fs()

In [2]:
import replica_utils

In [3]:
pd.set_option('display.max_columns', None)

In [24]:
gcs_path = "gs://calitp-analytics-data/data-analyses/big_data/MetroLink_LinkUS/"


In [5]:
analysis_result_names = [ "line_origin_union_station",
                         "riversideline_origin_montebello", "riversideline_origin_industry", "riversideline_origin_pomona",  
                         "riversideline_origin_ontario", "riversideline_origin_pedley",  "riversideline_origin_riverside",
                         
                         "sbline_origin_calstate_la", "sbline_origin_el_monte",  "sbline_origin_baldwin_park",
                         "sbline_origin_covina",  "sbline_origin_pomona_north", "sbline_origin_fontana",
                         "sbline_origin_claremont", "sbline_origin_montclair", "sbline_origin_upland", "sbline_origin_rancho_cucamonga", 
                         "sbline_origin_rialto", "sbline_origin_san_bernardino_depot", "sbline_origin_san_bernardino_dt" 
                        ]

In [6]:
# analysis_result_names_subset_test = ["sbline_origin_covina", "sbline_origin_calstate_la", 
#                                 "riversideline_origin_industry",  "riversideline_origin_montebello"]

In [74]:
results = []
for analysis in analysis_result_names:
    with get_fs().open(f"{gcs_path}replica_data_downloads/replica-socal_travel_analysis_{analysis}-trips_dataset.csv") as f:
        df = to_snakecase(pd.read_csv(f, dtype={25: str, 26:str}))
        
        gdf_stations = read_in_stations()
        station_col_map = dict(zip(gdf_stations['geoname'], gdf_stations['station_name_full']))
        station_line_map = dict(zip(gdf_stations['station_name_full'], gdf_stations['transit_lines']))
        
        df['origin_tract_station_area'] = df['origin_trct_2020'].map(station_col_map)
        df['destination_tract_station_area'] = df['destination_trct_2020'].map(station_col_map)
        
        station = analysis.split('_origin_')[1]  # Get the part after '_origin_'
        station_name = station.replace('_', ' ').upper()
        # line = analysis.split('_origin_')[0]
        
        auto_df = (df[df.primary_mode=="private_auto"])
        transit_df = (df[df.primary_mode=="public_transit"])
        
        metrolink_line = replica_utils.define_transit_line(line) 
        n_total_trips = len(df)
        n_private_auto_trips = len(auto_df)
        pct_private_auto_trips = ((len(auto_df)) / (len(df)))
        n_public_transit_trips = (len(transit_df))
        pct_public_transit_trips = ((len(transit_df)) / (len(df)))

        auto_mean_min, auto_median_min, auto_mean_miles, auto_median_miles = replica_utils.calc_auto_travel_info(df)
        transit_mean_min, transit_median_min, transit_mean_miles, transit_median_miles = replica_utils.calc_transit_travel_info(df)
        
        auto_top_name1, auto_top_num1, auto_top_name2, auto_top_num2, auto_top_name3, auto_top_num3 = replica_utils.get_top_and_bottom_tract_counts(auto_df, top_least = "top")
        auto_bottom_name1, auto_bottom_num1, auto_bottom_name2, auto_bottom_num2, auto_bottom_name3, auto_bottom_num3 = replica_utils.get_top_and_bottom_tract_counts(auto_df, top_least = "least")
        
        transit_top_name1, transit_top_num1, transit_top_name2, transit_top_num2, transit_top_name3, transit_top_num3 = replica_utils.get_top_and_bottom_tract_counts(transit_df, top_least = "top")
        transit_bottom_name1, transit_bottom_num1, transit_bottom_name2, transit_bottom_num2, transit_bottom_name3, transit_bottom_num3 = replica_utils.get_top_and_bottom_tract_counts(transit_df, top_least = "least")

        ## set up the table for all the results
        results.append({
                'station_name': station_name,
                # 'metrolink_line': metrolink_line,
                'total_trips': n_total_trips,
                'n_auto_trips_to_other_station_areas': n_private_auto_trips,
                'pct_auto_trips_to_other_station_areas': pct_private_auto_trips,
                'n_tranist_trips_to_other_station_areas': n_public_transit_trips,
                'pct_transit_trips_to_other_station_areas': pct_public_transit_trips,
                'auto_mean_minutes': auto_mean_min,
                'auto_median_minutes': auto_median_min,
                'auto_mean_miles': auto_mean_miles,
                'auto_median_miles': auto_median_miles,
                'transit_mean_minutes': transit_mean_min,
                'transit_median_minutes': transit_median_min,
                'transit_mean_miles': transit_mean_miles,
                'transit_median_miles': transit_median_miles,
            
                'auto_top_tract_traveled_to': auto_top_name1,
                'auto_top_tract_traveled_to_ntrips':auto_top_num1, 
                'auto_second_top_tract_traveled_to':auto_top_name2, 
                'auto_second_top_tract_traveled_to_ntrips':auto_top_num2, 
                'auto_third_top_tract_traveled_to':auto_top_name3, 
                'auto_third_top_tract_traveled_to_ntrips':auto_top_num3, 
                
                'auto_least_tract_traveled_to': auto_bottom_name1,
                'auto_least_tract_traveled_to_ntrips':auto_bottom_num1, 
                'auto_second_least_tract_traveled_to':auto_bottom_name2, 
                'auto_second_least_tract_traveled_to_ntrips':auto_bottom_num2, 
                'auto_third_least_tract_traveled_to':auto_bottom_name3, 
                'auto_third_least_tract_traveled_to_ntrips':auto_bottom_num3, 
                
                'transit_top_tract_traveled_to': transit_top_name1,
                'transit_top_tract_traveled_to_ntrips':transit_top_num1, 
                'transit_second_top_tract_traveled_to':transit_top_name2, 
                'transit_second_top_tract_traveled_to_ntrips':transit_top_num2, 
                'transit_third_top_tract_traveled_to':transit_top_name3, 
                'atransit_third_top_tract_traveled_to_ntrips':transit_top_num3, 
                
                'transit_least_tract_traveled_to': transit_bottom_name1,
                'transit_least_tract_traveled_to_ntrips':transit_bottom_num1, 
                'transit_second_least_tract_traveled_to':transit_bottom_name2, 
                'transit_second_least_tract_traveled_to_ntrips':transit_bottom_num2, 
                'transit_third_least_tract_traveled_to':transit_bottom_name3, 
                'atransit_third_least_tract_traveled_to_ntrips':transit_bottom_num3, 

                
                })

        
result_summary = pd.DataFrame(results)        

        

In [75]:
result_summary

,station_name,total_trips,n_auto_trips_to_other_station_areas,pct_auto_trips_to_other_station_areas,n_tranist_trips_to_other_station_areas,pct_transit_trips_to_other_station_areas,auto_mean_minutes,auto_median_minutes,auto_mean_miles,auto_median_miles,transit_mean_minutes,transit_median_minutes,transit_mean_miles,transit_median_miles,auto_top_tract_traveled_to,auto_top_tract_traveled_to_ntrips,auto_second_top_tract_traveled_to,auto_second_top_tract_traveled_to_ntrips,auto_third_top_tract_traveled_to,auto_third_top_tract_traveled_to_ntrips,auto_least_tract_traveled_to,auto_least_tract_traveled_to_ntrips,auto_second_least_tract_traveled_to,auto_second_least_tract_traveled_to_ntrips,auto_third_least_tract_traveled_to,auto_third_least_tract_traveled_to_ntrips,transit_top_tract_traveled_to,transit_top_tract_traveled_to_ntrips,transit_second_top_tract_traveled_to,transit_second_top_tract_traveled_to_ntrips,transit_third_top_tract_traveled_to,atransit_third_top_tract_traveled_to_ntrips,transit_least_tract_traveled_to,transit_least_tract_traveled_to_ntrips,transit_second_least_tract_traveled_to,transit_second_least_tract_traveled_to_ntrips,transit_third_least_tract_traveled_to,atransit_third_least_tract_traveled_to_ntrips
0,UNION STATION,18657,10339,0.554162,938,0.050276,31.680143,24.0,21.165722,14.3,78.638593,64.0,21.760768,21.10,Cal State LA,1185,Burbank - Downtown,1024,Montebello/Commerce,887,Solana Beach,1,San Bernardino - Depot,1,Redlands - Downtown,2,Glendale,78,El Monte,73,Commerce,68,Perris - Downtown,1,San Bernardino - Downtown,1,Redlands - University,1
1,MONTEBELLO,42917,22560,0.525666,655,0.015262,13.369238,7.0,7.194459,2.7,42.964885,30.0,7.341221,3.50,Montebello/Commerce,9740,Commerce,5357,Cal State LA,1519,Redlands - Esri,1,Vincent Grade/Acton,1,Old Town San Diego,1,Montebello/Commerce,367,Commerce,103,LA Union Station,51,San Bernardino - Downtown,1,Irvine,1,Upland,1
2,INDUSTRY,8588,5436,0.632976,51,0.005939,26.223142,23.0,17.304213,13.9,97.647059,82.0,26.864706,23.90,Covina,436,Montclair,383,Placentia,343,San Diego,1,Vincent Grade/Acton,1,Perris - Downtown,1,LA Union Station,16,Covina,3,Anaheim,2,Commerce,1,Santa Ana,1,Fullterton,1
3,POMONA,19454,10982,0.564511,186,0.009561,15.845110,11.0,9.885230,5.4,58.634409,47.0,13.086559,10.60,Montclair,2290,Claremont,963,Covina,481,Simi Valley,1,San Juan Capistrano,1,Burbank Airport,1,Covina,44,Montclair,16,LA Union Station,11,Redlands - University,1,Burbank - Downtown,1,Riverside - Downtown,1
4,ONTARIO,63621,35456,0.557300,74,0.001163,18.636084,16.0,11.136967,9.1,84.351351,82.5,17.785135,16.00,Fontana,2570,Riverside - La Sierra,2394,Montclair,1676,Via Princessa,1,Solana Beach,1,Burbank Airport,2,Montclair,11,Rialto,4,Upland,4,San Bernardino - Tippecanoe,1,Claremont,1,Moreno Valley / March Field,1
5,PEDLEY,8523,5720,0.671125,6,0.000704,21.623776,19.0,12.747238,9.8,89.500000,68.0,26.750000,19.70,Ontario - East,1235,Riverside - La Sierra,940,Riverside - Hunter Park/UCR,535,Solana Beach,1,Buena Park,1,Sylmar/San Fernando,1,LA Union Station,2,Riverside - Hunter Park/UCR,2,Fontana,1,Fontana,1,Riverside - Hunter Park/UCR,2,LA Union Station,2
6,RIVERSIDE,31087,19783,0.636375,236,0.007592,13.263762,11.0,8.226235,6.8,47.627119,34.5,7.964831,6.50,Riverside - Hunter Park/UCR,9247,Riverside - La Sierra,2295,Moreno Valley / March Field,2125,Northridge,1,Solana Beach,1,Irvine,1,Riverside - Hunter Park/UCR,118,Moreno Valley / March Field,43,Riverside - La Sierra,30,Santa Ana,1,Ontario - East,1,Fullterton,1
7,CALSTATE LA,9447,6282,0.664973,186,0.019689,20.878860,15.0,12.178383,6.7,62.935484,46.0,12.464516,7.45,LA Union Station,2185,Montebello/Commerce,1468,Commerce,693,San Bernardino - Depot,1,San Bernardino - Downtown,1,Redlands - University,1,LA Union Station,63,Montebello/Commerce,38,Commerce,24,Orange,1,Sun Valley,1,Industry,1
8,EL MONTE,9072,6596,0.727072,162,0.017857,24.060794,19.0,15.411810,10.7,69.302469,60.0,16.438272,14.30,Montebello/Commerce,947,LA Union Station,459,Covina,363,San Cleme

In [ ]:
## export
# result_summary.to_csv("MetroLink_Stations_Replica_Summaries.csv")

In [ ]:
# df_auto.columns

In [ ]:
# df.primary_mode.value_counts()

In [ ]:
#### add in which station area is most traveled to

In [8]:
## starting with one station
with get_fs().open(f"{gcs_path}replica_data_downloads/replica-socal_travel_analysis_sbline_origin_covina-trips_dataset.csv") as f:
        df = to_snakecase(pd.read_csv(f, dtype={25: str, 26:str}))

In [9]:
df.sample()

,activity_id,origin_bgrp_2020,origin_trct_2020,origin_cty_2020,origin_st_2020,destination_bgrp_2020,destination_trct_2020,destination_cty_2020,destination_st_2020,primary_mode,trip_purpose,previous_trip_purpose,trip_start_time,trip_end_time,trip_duration_minutes,trip_distance_miles,vehicle_type,vehicle_fuel_type,transit_submode,transit_agency,transit_route,origin_land_use,origin_building_use,destination_land_use,destination_building_use,trip_taker_person_id,trip_taker_household_id,trip_taker_age,trip_taker_sex,trip_taker_race_ethnicity,trip_taker_employment_status,trip_taker_wfh,trip_taker_individual_income,trip_taker_commute_mode,trip_taker_household_size,trip_taker_household_income,trip_taker_available_vehicles,trip_taker_resident_type,trip_taker_industry,trip_taker_building_type,trip_taker_school_grade_attending,trip_taker_education,trip_taker_tenure,trip_taker_language,trip_taker_home_bgrp_2020,trip_taker_home_trct_2020,trip_taker_home_cty_2020,trip_taker_home_st_2020,trip_taker_work_bgrp_2020,trip_taker_work_trct_2020,trip_taker_work_cty_2020,trip_taker_work_st_2020
0,3003580386741640042,"2 (Tract 4061.01, Los Angeles, CA)","4061.01 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 4033.28, Los Angeles, CA)","4033.28 (Los Angeles, CA)","Los Angeles County, CA",California,commercial,commercial,commercial,13:29:03,13:44:06,15,8.2,medium_commercial,unknown_fuel_type,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12179869879852593129,"1 (Tract 4061.01, Los Angeles, CA)","4061.01 (Los Angeles, CA)","Los Angeles County, CA",California,"1 (Tract 4016.02, Los Angeles, CA)","4016.02 (Los Angeles, CA)","Los Angeles County, CA",California,commercial,commercial,commercial,17:45:42,18:05:29,19,10.9,medium_commercial,unknown_fuel_type,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1622094024392114287,"1 (Tract 4037.22, Los Angeles, CA)","4037.22 (Los Angeles, CA)","Los Angeles County, CA",California,"2 (Tract 5323.04, Los Angeles, CA)","5323.04 (Los Angeles, CA)","Los Angeles County, CA",California,commercial,commercial,commercial,12:37:07,13:11:05,33,22.2,heavy_commercial,unknown_fuel_type,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6603655617372776162,"2 (Tract 4060, Los Angeles, CA)","4060 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 4053.02, Los Angeles, CA)","4053.02 (Los Angeles, CA)","Los Angeles County, CA",California,commercial,commercial,commercial,11:38:11,11:45:44,7,3.7,medium_commercial,unknown_fuel_type,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7947274080200339741,"4 (Tract 4037.21, Los Angeles, CA)","4037.21 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 9200.13, Los Angeles, CA)","9200.13 (Los Angeles, CA)","Los Angeles County, CA",California,commercial,commercial,commercial,15:54:38,17:05:56,71,52.8,medium_commercial,unknown_fuel_type,NaN,NaN,NaN,unknown,unknown,unknown,unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9472,2275166492827802676,"2 (Tract 4061.01, Los Angeles, CA)","4061.01 (Los Angeles, CA)","Los Angeles County, CA",California,"3 (Tract 4323, Los Angeles, CA)","4323 (Los Angeles, CA)","Los Angeles County, CA",California,auto_passenger,lodging,recreation,17:57:00,18:14:43,17,11.3,unknown_vehicle_type,unknown_fuel_type,NaN,NaN,NaN,office,office,mixed_us

In [10]:
### mapping the stations names to the census tracts

In [11]:
all_stations_list = ["lossan-simi-valley", "lossan-burbank-airport", "lossan-oxnard",  "lossan-camarillo", "lossan-moorpark",  "lossan-chatsworth",
                     "lossan-northridge", "lossan-van-nuys", "lossan-burbank", "lossan-glendale", 
                     "lossan-commerce", "lossan-norwalk", "lossan-buena-park", "lossan-fullerton", "lossan-anaheim",
                     "lossan-orange", "lossan-santa-ana", "lossan-tustin", "lossan-irvine", "lossan-laguna-niguel", 
                     "lossan-san-juan-capistrano", "lossan-san-clemente", "lossan-san-clemente-pier", "lossan-oceanside", 
                     "lossan-solana-beach", "lossan-san-diego", "lossan-ot-san-diego", "lossan-union-station",
                     
                     "riverside-line-montebello-commerce", "riverside-line-industry", "riverside-line-pomona",
                     "riverside-line-ontario","riverside-line-pedley",  "riverside-line-riverside",
                     
                     "sb-line-cal-state-la", "sb-line-el-monte",  "sb-line-baldwin-park", "sb-line-covina",  "sb-line-pomona-north",
                     "sb-line-claremont", "sb-line-montclair", "sb-line-upland", "sb-line-rancho-cucamonga", "sb-line-fontana", 
                     "sb-line-rialto", "sb-line-san-bernardino-depot", "sb-line-san-bernardino-dt", 
                     "sb-line-redlands-dt", "sb-line-redlands-esri", "sb-line-redlands-univ", "sb-line-san-bernardino-tippecanoe",
                     
                     "91-perris-line-corona", "91-perris-line-corona-west", "91-perris-line-moreno-valley-march", 
                     "91-perris-line-perris-south", "91-perris-line-perris-downtown", "91-perris-line-placentia-future", 
                     "91-perris-line-riverside-hunter-park", "91-perris-line-riverside-la-sierra",
                     
                     "antelope-valley-burbank-airport-n", "antelope-valley-lancaster", "antelope-valley-newhall", "antelope-valley-palmdale",
                     "antelope-valley-santa-clarita", "antelope-valley-sun-valley", "antelope-valley-sylmar-san-fernando",
                     "antelope-valley-via-princessa", "antelope-valley-vincent-grade", "antelope-valley-vista-canyon",
                     
                     "inland-empire-oc-anaheim-canyon", 
                     
                     "ventura-line-ventura-east", "ventura-line-ventura-dtwn",                    
                    ]   

In [73]:
test_all_stations.sample()

,geoid,geoname,alias,centrlat,centrlon,geometry,station_name,station_name_full,transit_lines
348,3641019,"427.31 (Riverside, CA)",None,33.7378,-117.2199,"POLYGON ((-117.26330 33.71461, -117.26361 33.7...",91-perris-line-perris-south,Perris - South,91/Perris Valley Line


,geoid,geoname,alias,centrlat,centrlon,geometry,station_name,station_name_full,transit_lines
0,3612448,"83.02 (Ventura, CA)",None,34.2763,-118.6730,"POLYGON ((-118.68274 34.28135, -118.68274 34.2...",lossan-simi-valley,Simi Valley,"Ventura County Line, Amtrak Pacific Surfliner"
1,3632207,"83.07 (Ventura, CA)",None,34.2767,-118.6870,"POLYGON ((-118.69138 34.28149, -118.69138 34.2...",lossan-simi-valley,Simi Valley,"Ventura County Line, Amtrak Pacific Surfliner"
2,3633654,"80.02 (Ventura, CA)",None,34.2687,-118.7266,"POLYGON ((-118.74785 34.26616, -118.74786 34.2...",lossan-simi-valley,Simi Valley,"Ventura County Line, Amtrak Pacific Surfliner"
3,3646274,"83.04 (Ventura, CA)",None,34.2672,-118.6585,"POLYGON ((-118.67636 34.26690, -118.67635 34.2...",lossan-simi-valley,Simi Valley,"Ventura County Line, Amtrak Pacific Surfliner"
4,3653763,"82.01 (Ventura, CA)",None,34.2768,-118.7045,"POLYGON ((-118.71775 34.28164, -118.71775 34.2...",lossan-simi-valley,Simi Valley,"Ventura County Line, Amtrak Pacific Surfliner"
...,...,...,...,...,...,...,...,...,...
492,3646035,"23.01 (Ventura, CA)",None,34.2905,-119.2995,"POLYGON ((-119.30352 34.28136, -119.30336 34.2...",ventura-line-ventura-dtwn,Ventura - Downtown / Beach,Amtrak Pacific Surfliner
493,3670973,"95 (Ventura, CA)",None,34.3650,-119.3572,"MULTIPOLYGON (((-119.44606 34.34783, -119.4460...",ventura-line-ventura-dtwn,Ventura - Downtown / Beach,Amtrak Pacific Surfliner
494,3671720,"24 (Ventura, CA)",None,34.2765,-119.2945,"POLYGON ((-119.30858 34.28136, -119.30868 34.2...",ventura-line-ventura-dtwn,Ventura - Downtown / Beach,Amtrak Pacific Surfliner
495,3672846,"22 (Ventura, CA)",None,34.2996,-119.2884,"POLYGON ((-119.30075 34.28104, -119.30051 34.2...",ventura-line-ventura-dtwn,Ventura - Downtown / Beach,Amtrak Pacific Surfliner


In [ ]:
df.destination_tract_station_area.value_counts()